## Imports

In [11]:
import gc
import logging
import math
import random
import time
import warnings
from collections import defaultdict
from datetime import datetime
from importlib import reload
from itertools import product
from subprocess import Popen, PIPE
from threading import Thread
from warnings import catch_warnings, simplefilter

import matplotlib.pyplot as plt
import numpy as np
import torch
from scipy.stats import qmc, norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import accuracy_score
from torch.autograd import Variable
from torch.nn import *
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm

reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')

## Neural Network Classes

In [12]:
class SimpleCNN(Module):
    def __init__(self, config, verbose):
        super(SimpleCNN, self).__init__()
        img_dim = config['shape'][2]
        channels = config['shape'][1]
        conv_layers = np.array([
            [LazyConv2d(channels * 3 ** (depth + 1), kernel_size=3, stride=1, padding=1),
             LazyBatchNorm2d(),
             ReLU(inplace=True),
             MaxPool2d(kernel_size=2, stride=2)]
            for depth in range(config['factors'].get('nr_conv_layers', 2))
        ]).flatten().tolist()
        linear_layers = [
            LazyLinear(max(math.floor(channels * img_dim ** 2 / 2 ** (depth + 1)), channels)) for depth in
            range(config['factors'].get('nr_linear_layers', 3) - 1)
        ]
        self.layers = Sequential(
            *conv_layers,
            Flatten(start_dim=1),
            *linear_layers,
            LazyLinear(config['labels'])
        )
        if verbose:
            logging.info(self.layers)

    def forward(self, x):
        x = self.layers(x)
        return x


class DensePolyNN(Module):
    def __init__(self, config, verbose):
        super(DensePolyNN, self).__init__()

        img_dim = config['shape'][2]
        channels = config['shape'][1]
        linear_layers = [
            LazyLinear(max(math.floor(channels * img_dim ** 2 / 2 ** (depth + 1)), channels)) for depth in
            range(config['factors'].get('nr_linear_layers', 3) - 1)
        ]

        self.layers = Sequential(
            Flatten(start_dim=1),
            *linear_layers,
            LazyLinear(config['labels'])
        )
        if verbose:
            logging.info(self.layers)

    def forward(self, x):
        x = self.layers(x)
        return x


class DenseLinearNN(Module):
    def __init__(self, config, verbose):
        super(DenseLinearNN, self).__init__()

        img_dim = config['shape'][2]
        channels = config['shape'][1]
        linear_layers = [
            LazyLinear(max(math.floor(
                channels * img_dim ** 2 - (depth + 1) * (
                        channels * img_dim ** 2 / config['factors'].get('nr_linear_layers', 3))),
                channels)) for depth in range(config['factors'].get('nr_linear_layers', 3) - 1)]

        self.layers = Sequential(
            Flatten(start_dim=1),
            *linear_layers,
            LazyLinear(config['labels'])
        )
        if verbose:
            logging.info(self.layers)

    def forward(self, x):
        x = self.layers(x)
        return x

## Get GPU values

In [13]:
class GPUMonitor(Thread):
    def __init__(self, delay):
        super(GPUMonitor, self).__init__()
        self.delay = delay
        self.power_readings = []
        self.running = True
        self.start()

    def run(self):
        while self.running:
            try:
                p = Popen('nvidia-smi --query-gpu=power.draw --format=csv,noheader,nounits'.split(' '), stdout=PIPE)
                stdout, stderror = p.communicate()
                self.power_readings.append(float(stdout.strip()))
                p.terminate()
            except:
                logging.error('Something went wrong while retrieving GPU readings...')
            time.sleep(self.delay)

    def stop(self):
        self.running = False

    def reset_energy(self):
        self.power_readings = []

    def get_power_average(self):
        return np.mean(self.power_readings)


## Model operations

In [14]:
def create_model(net, config, verbose):
    model = net(config=config, verbose=verbose)
    optimizer = Adam(model.parameters(),
                     lr=config['factors'].get('learning_rate', 1e-3),
                     betas=(config['factors'].get('beta1', 0.9), config.get('beta2', 0.999)),
                     eps=config['factors'].get('epsilon', 1e-8),
                     weight_decay=config['factors'].get('weight_decay', 0)
                     )
    criterion = CrossEntropyLoss()
    if torch.cuda.is_available():
        if verbose:
            logging.info('Using GPU')
        model = model.cuda()
        criterion = criterion.cuda()
    return model, optimizer, criterion


def train(net, config, batches, tag, verbose=False):
    logging.info(f'Using {config["factors"]}')
    logging.info(f'Constructing {tag}')
    model, optimizer, criterion = create_model(net, config, verbose)
    train_losses = []
    logging.info('Training the model:')
    t_start = time.perf_counter()
    gpu_monitor.reset_energy()
    for _ in tqdm(range(config['epochs'])):
        for batch_id, batch in batches:
            train_x = batch[0]
            train_y = batch[1]
            model.train()
            train_x, train_y = Variable(train_x), Variable(train_y)
            if torch.cuda.is_available():
                train_x = train_x.cuda()
                train_y = train_y.cuda()

            # clearing the Gradients of the model parameters
            optimizer.zero_grad()

            # prediction for training set
            output_train = model(train_x)

            # computing the training loss
            loss_train = criterion(output_train, train_y)
            train_losses.append(loss_train.item())

            # computing the updated weights of all the model parameters
            loss_train.backward()
            optimizer.step()
    avg_power_ = gpu_monitor.get_power_average()
    duration_ = time.perf_counter() - t_start
    if verbose:
        logging.info(f'{avg_power_=}, {duration_=}')
        plt.plot(train_losses, label='Training loss')
        plt.yscale('log')
        plt.legend()
        plt.show()

    return model, tag, config['factors'], avg_power_, duration_


def test(models, test_x, test_y):
    logging.info('Generating predictions and calculating accuracy')
    test_results = []
    for model, tag, factors, avg_power, duration in models:
        with torch.no_grad():
            output = model(test_x.cuda())

        softmax = torch.exp(output).cpu()
        prob = list(softmax.numpy())
        predictions = np.argmax(prob, axis=1)
        accuracy = accuracy_score(test_y, predictions)
        test_results.append((tag, accuracy, factors, avg_power, duration))
        logging.info(f'{tag}: {accuracy=}')
    return test_results


def predict(model, test_x, predictions):
    logging.info('Generating predictions')
    with torch.no_grad():
        output = model(test_x.cuda())

    softmax = torch.exp(output).cpu()
    prob = list(softmax.numpy())
    predictions['label'] = np.argmax(prob, axis=1)
    return predictions

## Training strategies

In [15]:
def random_strategy(factors_bounds, random_function, tag):
    """
    Returns a configuration of (hyper)parameter values according to a random strategy
    :param factors_bounds: The lower and upper bound for all the values
    :param random_function: The method for generating random samples
    :param tag: NOT USED, PASS NONE VALUE
    :return: (hyper)parameter configuration
    """
    logging.info('Applying the random strategy...')

    factors = []
    lowers = []
    uppers = []
    is_ints = []
    for factor, (low, up, is_int) in factors_bounds.items():
        factors.append(factor)
        lowers.append(low)
        uppers.append(up)
        is_ints.append(is_int)

    return random_function(factors, 1, lowers, uppers, is_ints)[0]


def grid_search(factors_bounds, random_function, tag):
    """
    Returns a configuration of (hyper)parameter values according to a grid search strategy
    :param factors_bounds: The lower and upper bound for all the values
    :param random_function: NOT USED, PASS NONE VALUE
    :param tag: The tag of the neural network
    :return: (hyper)parameter configuration
    """
    logging.info('Applying a grid search strategy...')
    if tag not in grid_search_space:
        all_values = []
        size = 1
        real_value_split = 5
        for factor, (lower, upper, is_int) in factors_bounds.items():
            if is_int:
                all_values.append([lower + i for i in range(upper - lower)])
                size *= (upper - lower)
            else:
                all_values.append([lower + i * (upper - lower) / real_value_split for i in range(real_value_split)])
                size *= real_value_split
        search_space = list(product(*all_values))
        random_indices = random.sample(range(size), config['experiments'])
        grid_search_space[tag] = (random_indices, search_space)

    random_indices, search_space = grid_search_space[tag]
    factors = list(factors_bounds.keys())
    random_idx = random_indices.pop()
    return {factors[i]: search_space[random_idx][i] for i in range(len(factors))}


def bayesian_pi(factor_bounds, random_function, tag):
    """
    Returns a configuration of (hyper)parameter values according to a Bayesian strategy with probability of improvement
    :param factors_bounds: The lower and upper bound for all the values
    :param random_function: The method for generating random samples
    :param tag: The tag of the neural network
    :return: (hyper)parameter configuration
    """
    logging.info('Applying the bayesian strategy with probability of improvement...')

    factors = []
    lowers = []
    uppers = []
    is_ints = []
    for factor, (low, up, is_int) in factor_bounds.items():
        factors.append(factor)
        lowers.append(low)
        uppers.append(up)
        is_ints.append(is_int)

    # Return a random sample and initialize the regression model if it does not exist
    if tag not in regression_models:
        random_factors = random_function(factors, 1, lowers, uppers, is_ints)[0]
        regression_models[tag] = {
            "model": GaussianProcessRegressor(),
            "X": [],
            "y": []
        }
        return random_factors

    # Generate 10000 random samples and return the one with the highest probability of improvement, update the regression model
    regression_models[tag]['model'].fit(regression_models[tag]['X'], regression_models[tag]['y'])
    random_samples = random_function(factors, 10000, lowers, uppers, is_ints)
    candidate_sample = best_probability_of_improvement(factors, random_samples, regression_models[tag]['model'],
                                                       max(regression_models[tag]['y']))
    return candidate_sample

## Random functions

In [16]:
def quasi_random(factors, length, lower, upper, is_int, sampler=qmc.Halton):
    sampler = sampler(len(factors))
    sample = sampler.random(length)
    random_samples = qmc.scale(sample, lower, upper)
    return [{factors[i]: round(s[i]) if is_int[i] else s[i] for i in range(len(factors))} for s in random_samples]


def uniform_random(factors, length, lower, upper, is_int):
    return [
        {factors[i]: round(random.uniform(lower[i], upper[i])) if is_int[i] else random.uniform(lower[i], upper[i]) for
         i in range(len(factors))} for _ in range(length)]

## Acquisition functions

In [17]:
def best_probability_of_improvement(factors, samples, model, y_best, maximize=True):
    best = -1
    x_next = None
    for x in samples:
        x = list(x.values())
        with catch_warnings():
            # ignore generated warnings
            simplefilter("ignore")
            mu_, std_ = model.predict([x], return_std=True)
            pi = norm.cdf((mu_ - y_best) / (std_ + 1e-9)) if maximize else norm.cdf((y_best - mu_) / (std_ + 1e-9))
            if pi > best:
                best = pi
                x_next = x
    return {factors[i]: x_next[i] for i in range(len(factors))}

## Configuration

In [18]:
# Configurable parameters:
# nr_linear_layers: default 3
# nr_conv_layers: default 2
# learning_rate: default 1e-3
# beta1: default 0.9
# beta2: default 0.999
# epsilon: default 1e-8
# weight_decay: default 0
config = {
    'experiments': 5,
    'repetitions': 1,
    'epochs': 10,
    'batch_size': 512,
    'dataset': datasets.FashionMNIST,
    'random_function': quasi_random,
    'optimization_strategy': bayesian_pi,
    'factor_bounds': {
        'nr_linear_layers': (1, 8, True),
        'learning_rate': (1e-5, 5e-3, False),
        'beta1': (0.4, 1, False),
        'beta2': (0.9, 1, False),
        'epsilon': (1e-9, 1e-7, False),
        'weight_decay': (0, 0.1, False)
    }
}

## Load data

In [19]:
logging.info('Loading data...')
training_data = config['dataset'](
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)
train_dataloader = DataLoader(training_data, batch_size=config['batch_size'], shuffle=True)
batches = [(batch_id, batch) for batch_id, batch in enumerate(train_dataloader)]
logging.info('Training data loaded')

test_data = config['dataset'](
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)
test_dataloader = DataLoader(test_data, batch_size=len(test_data), shuffle=False)
test_x, test_y = next(iter(test_dataloader))
logging.info('Test data loaded')
config['shape'] = batches[0][1][0].shape
config['labels'] = len(training_data.classes)

03:19:26 INFO: Loading data...
03:19:31 INFO: Training data loaded
03:19:32 INFO: Test data loaded


## Train models

In [22]:
seed = 45
torch.manual_seed(seed)
random.seed(seed)
results = defaultdict(list)
regression_models = {}
grid_search_space = {}
gpu_monitor = GPUMonitor(0.1)
####################
nets_to_train = [DensePolyNN, SimpleCNN]
####################
for experiment in range(config['experiments']):
    logging.info(f'\n\n#### RUNNING EXPERIMENT {experiment + 1}/{config["experiments"]} ####\n')
    models = []
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for net in nets_to_train:
            config['factors'] = config['optimization_strategy'](config['factor_bounds'], config['random_function'],
                                                                net.__name__)
            for _ in range(config['repetitions']):
                t_start = time.perf_counter()
                models.append(train(DensePolyNN, config, batches, net.__name__))

    test_results = test(models, test_x, test_y)
    for tag, acc, used_config, avg_power, duration in test_results:
        if tag in regression_models:
            regression_models[tag]['X'].append(list(used_config.values()))
            regression_models[tag]['y'].append(acc)
        results[tag].append((experiment, acc, used_config, avg_power, duration))
    gc.collect()
    torch.cuda.empty_cache()
gpu_monitor.stop()

03:21:28 INFO: 

#### RUNNING EXPERIMENT 1/5 ####

03:21:28 INFO: Applying the bayesian strategy with probability of improvement...
03:21:28 INFO: Using {'nr_linear_layers': 1, 'learning_rate': 0.004209464312628585, 'beta1': 0.7141144693328179, 'beta2': 0.9692403335171381, 'epsilon': 1.5842192434657596e-08, 'weight_decay': 0.020649188632317243}
03:21:28 INFO: Constructing DensePolyNN
03:21:28 INFO: Training the model:
100%|██████████| 10/10 [00:02<00:00,  4.33it/s]
03:21:30 INFO: Applying the bayesian strategy with probability of improvement...
03:21:30 INFO: Using {'nr_linear_layers': 7, 'learning_rate': 0.002364574628503909, 'beta1': 0.7031980723821807, 'beta2': 0.9772449481299729, 'epsilon': 7.478416170828334e-08, 'weight_decay': 0.048183319524328645}
03:21:30 INFO: Constructing SimpleCNN
03:21:30 INFO: Training the model:
100%|██████████| 10/10 [00:04<00:00,  2.04it/s]
03:21:35 INFO: Generating predictions and calculating accuracy
03:21:35 INFO: DensePolyNN: accuracy=0.7984
03:21:3

## Write Results

In [23]:
file_name = f'exp_results_{datetime.today().strftime("%d%m%Y_%H%M%S")}.csv'
file = open(f'results/{file_name}', 'a')
for tag in results:
    print(f'{tag}:')
    file.write(f'{tag}:\n')
    print(f'experiment,accuracy,avg_power,duration,{"".join(key + "," for key in results[tag][0][2])[:-1]}')
    file.write(f'experiment,accuracy,avg_power,duration,{"".join(key + "," for key in results[tag][0][2])[:-1]}\n')
    [print(f'{experiment},{acc},{avg_power},{duration},{"".join(str(val) + "," for val in used_config.values())[:-1]}')
     for
     experiment, acc, used_config, avg_power, duration in results[tag]]
    file.writelines(
        [f'{experiment},{acc},{avg_power},{duration},{"".join(str(val) + "," for val in used_config.values())[:-1]}\n'
         for
         experiment, acc, used_config, avg_power, duration in results[tag]])
file.close()
logging.info(f'Results logged to: results/{file_name}')

03:22:53 INFO: Results logged to: results/exp_results_27122021_152253.csv


DensePolyNN:
experiment,accuracy,avg_power,duration,nr_linear_layers,learning_rate,beta1,beta2,epsilon,weight_decay
0,0.7984,63.27333333333332,2.3113186379996478,1,0.004209464312628585,0.7141144693328179,0.9692403335171381,1.5842192434657596e-08,0.020649188632317243
1,0.7927,63.654545454545456,1.9702858439995907,1,0.004352411593993898,0.7170002497208019,0.9734948786433103,1.702100001426253e-08,0.02463118340287719
2,0.826,63.285833333333336,2.3259776719987713,1,0.0015441282035786193,0.6998102425176547,0.9418235719113304,6.058995015082956e-09,0.0053449788238321095
3,0.8073,50.169000000000004,1.9853815659989777,1,0.0019305403740994,0.6706524304230879,0.9067077979119442,9.082539987981628e-09,0.020669383351720097
4,0.8282,63.053999999999995,1.9415338979997614,1,0.0018846035875471566,0.7018373277067391,0.9430680956169735,2.3003093424082845e-08,0.003588750352586734
SimpleCNN:
experiment,accuracy,avg_power,duration,nr_linear_layers,learning_rate,beta1,beta2,epsilon,weight_decay
0,0.0991,77.450